# Notebook 2
The objective for this notebook is used to build the feature pipeline and encoder using Scikit learn. It also aim to use a different encoding technique (hashing encoding) to reduce the dimension of the input feature (brewery_name) and see if it improve overall performance.


Benny Lee - 13371063

In [1]:
%load_ext autoreload
%autoreload 2

# Load packages and data

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

In [3]:
df = pd.read_csv('../data/raw/beer_reviews.csv')

# Prepare Data

In [4]:
# Only copy these features that are relevant to the api
df_cleaned = df[['brewery_name','review_aroma','review_appearance','review_palate','review_taste','beer_abv','beer_style']].copy()

df_cleaned.columns

Index(['brewery_name', 'review_aroma', 'review_appearance', 'review_palate',
       'review_taste', 'beer_abv', 'beer_style'],
      dtype='object')

**Drop rows contain num value**

In [5]:
df_cleaned.dropna(inplace=True)
df_cleaned.isnull().sum()

brewery_name         0
review_aroma         0
review_appearance    0
review_palate        0
review_taste         0
beer_abv             0
beer_style           0
dtype: int64

In [6]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 102943
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   brewery_name       100000 non-null  object 
 1   review_aroma       100000 non-null  float64
 2   review_appearance  100000 non-null  float64
 3   review_palate      100000 non-null  float64
 4   review_taste       100000 non-null  float64
 5   beer_abv           100000 non-null  float64
 6   beer_style         100000 non-null  object 
dtypes: float64(5), object(2)
memory usage: 6.1+ MB


**Split data into training, validation and test sets**

In [7]:
from src.data.sets import split_sets_random, save_sets

col_target='beer_style'

X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(df_cleaned, target_col=col_target, test_ratio=0.2, to_numpy=False)

save_sets(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test, path='../data/processed/')

In [8]:
X_train.shape

(60000, 6)

**Create pipeline to transform input features**

In [9]:
col_category=['brewery_name']
col_numeric=['review_aroma', 'review_appearance', 'review_palate', 'review_taste', 'beer_abv']

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer
import joblib
from joblib import dump
from category_encoders.hashing import HashingEncoder

pipeline_transform = Pipeline([
    ('encoder', HashingEncoder(cols=['brewery_name'])),
    ('scaler', StandardScaler())
])

X_train=pipeline_transform.fit_transform(X_train)
X_val=pipeline_transform.transform(X_val)
X_test=pipeline_transform.transform(X_test)

# Save the feature pipeline
# dump(pipeline_transform,  '../../api/models/pipeline_features.joblib')

In [10]:
X_train

array([[ 1.5224487 , -0.34479141, -0.19477316, ...,  1.02239645,
         1.5886256 ,  1.3702044 ],
       [ 1.5224487 , -0.34479141, -0.19477316, ..., -0.5293537 ,
         0.1410648 ,  0.94337641],
       [-0.65683658, -0.34479141, -0.19477316, ...,  0.24652137,
         0.1410648 , -0.80661831],
       ...,
       [ 1.5224487 , -0.34479141, -0.19477316, ...,  0.24652137,
         0.8648452 , -0.33710753],
       [-0.65683658,  2.90030424, -0.19477316, ...,  1.02239645,
         0.8648452 ,  1.15679041],
       [-0.65683658, -0.34479141, -0.19477316, ...,  1.79827153,
         0.8648452 ,  1.3275216 ]])

**Encode target variable**

In [11]:
target_encoder = LabelEncoder()

y_train = target_encoder.fit_transform(y_train)
y_val = target_encoder.transform(y_val)
y_test = target_encoder.transform(y_test)

# Save the targer encoder
# joblib.dump(target_encoder, '../../api/models/encoder_target.joblib')

#target_encoder.inverse_transform([10])
#target_encoder.transform(['American IPA'])

In [12]:
from src.models.pytorch import PytorchDataset

train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)

# Define Model architecture

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [14]:
from src.models.pytorch import PytorchMultiClass
model = PytorchMultiClass(X_train.shape[1])

In [15]:
from src.models.pytorch import get_device

device = get_device()
model.to(device)

PytorchMultiClass(
  (layer_1): Linear(in_features=13, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=104, bias=True)
  (softmax): Softmax(dim=1)
)

In [16]:
# Print the architecture of model
print(model)

PytorchMultiClass(
  (layer_1): Linear(in_features=13, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=104, bias=True)
  (softmax): Softmax(dim=1)
)


# Train model

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
N_EPOCHS = 30
BATCH_SIZE = 32

In [18]:
from src.models.pytorch import test_classification, train_classification

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(val)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')


Epoch: 0
	(train)	|	Loss: 0.1392	|	Acc: 22.1%
	(val)	|	Loss: 0.1377	|	Acc: 25.9%
Epoch: 1
	(train)	|	Loss: 0.1375	|	Acc: 26.8%
	(val)	|	Loss: 0.1370	|	Acc: 27.9%
Epoch: 2
	(train)	|	Loss: 0.1368	|	Acc: 29.2%
	(val)	|	Loss: 0.1360	|	Acc: 31.4%
Epoch: 3
	(train)	|	Loss: 0.1363	|	Acc: 30.9%
	(val)	|	Loss: 0.1357	|	Acc: 32.8%
Epoch: 4
	(train)	|	Loss: 0.1360	|	Acc: 32.0%
	(val)	|	Loss: 0.1354	|	Acc: 33.8%
Epoch: 5
	(train)	|	Loss: 0.1357	|	Acc: 32.8%
	(val)	|	Loss: 0.1352	|	Acc: 33.9%
Epoch: 6
	(train)	|	Loss: 0.1355	|	Acc: 33.2%
	(val)	|	Loss: 0.1351	|	Acc: 33.9%
Epoch: 7
	(train)	|	Loss: 0.1354	|	Acc: 33.4%
	(val)	|	Loss: 0.1351	|	Acc: 34.2%
Epoch: 8
	(train)	|	Loss: 0.1353	|	Acc: 33.8%
	(val)	|	Loss: 0.1350	|	Acc: 34.3%
Epoch: 9
	(train)	|	Loss: 0.1353	|	Acc: 33.8%
	(val)	|	Loss: 0.1349	|	Acc: 34.7%
Epoch: 10
	(train)	|	Loss: 0.1352	|	Acc: 34.1%
	(val)	|	Loss: 0.1348	|	Acc: 34.9%
Epoch: 11
	(train)	|	Loss: 0.1351	|	Acc: 34.4%
	(val)	|	Loss: 0.1348	|	Acc: 34.9%
Epoch: 12
	(train)	|	Loss:

In [19]:
# Save model
#torch.save(model, "../../api/models/leeb_pytorch.pt")

# Test Model

In [20]:
test_loss, test_acc = test_classification(test_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)
print(f'\t(Test)\t|\tLoss: {test_loss:.4f}\t|\tAcc: {test_acc * 100:.1f}%')

	(Test)	|	Loss: 0.1340	|	Acc: 37.3%


# Test API code

In [21]:
model = torch.load('../../api/models/pytorch.pt')
pipeline_features = joblib.load('../../api/models/pipeline_features.joblib')
encoder_target = joblib.load('../../api/models/encoder_target.joblib')

def format_features(brewery_name: float, 
                    review_aroma: float, 
                    review_appearance: float, 
                    review_palate: float, 
                    review_taste: float, 
                    beer_abv: float):
  return {
  	'brewery_name': [brewery_name],
    'review_aroma': [review_aroma],
    'review_appearance': [review_appearance],
    'review_palate': [review_palate],
    'review_taste': [review_taste],
    'beer_abv': [beer_abv]
    }

dict_features = format_features("Vecchio Birraio", 1.0, 1.0, 1.0, 1.0, 1.0 )

df_inputs = pd.DataFrame(dict_features)
    
# Encoding input features
df_transformed=pipeline_features.transform(df_inputs)
    
device= torch.device('cpu')
    
df_tensor = torch.Tensor(np.array(df_transformed)).to(device)
    
pred_beer_style = model(df_tensor).argmax(1)

result = encoder_target.inverse_transform(pred_beer_style.tolist())[0]


In [22]:
from fastapi import FastAPI, Query
from starlette.responses import JSONResponse

JSONResponse(result)

# Save the model, features pipeline and target encoder

In [23]:
# Save model
torch.save(model, "../models/leeb_pytorch.pt")

# Save the targer encoder
dump(target_encoder, '../models/leeb_encoder_target.joblib')

# Save the feature pipeline
dump(pipeline_transform,  '../models/leeb_pipeline_features.joblib')

['../models/leeb_pipeline_features.joblib']